In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

In [58]:
#please don't use `list` as a variable name.
file_list = [#'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-01.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-02.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-03.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-04.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-05.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-06.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-07.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-08.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-09.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-10.csv',
             #'s3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-11.csv',
             's3://nyc-tlc-trip-2018/nyc-2018-yellow/yellow_tripdata_2018-12.csv']

def yield_dfs(file_list):
    """generator function to yield dataframes."""
    for file_name in file_list:
        df = pd.read_csv(file_name)
        # you may be able to reduce the memory requirements by doing some pre-processing of the dataframe here. e.g. convert strings to booleans so save memore.
        yield df

df = pd.concat(yield_dfs(file_list))

In [59]:
df_taxi_zones = pd.read_csv('taxi+_zone_lookup.csv',engine='python')

In [60]:
def remove_NaN(df):
    
    #Remove NaN rows
    #threshold = int((percentage)*df.shape[0])
    #df = df.dropna(axis=0, thresh = threshold)
    
    #Remove Unknown location IDs
    PU_NaN_condition = df[df['PULocationID'].isin(['264', '265'])].index
    df = df.drop(PU_NaN_condition)
    
    DO_NaN_condition = df[df['DOLocationID'].isin(['264', '265'])].index
    df = df.drop(DO_NaN_condition)
    
    return df

df = remove_NaN(df)

In [61]:
clean_up_dict = {'df':df, 'NaN_percentage':0.75, 
                 'tpep_PU':df['tpep_pickup_datetime'], 'tpep_DO':df['tpep_dropoff_datetime'],
                'neg_features':['trip_distance', 'trip_duration', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge'],
                 'cat_features':['RatecodeID', 'payment_type'], #'store_and_fwd_flag'
                'drop_features':['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'fare_amount', 'extra', 'mta_tax', 'improvement_surcharge'],
                 'taxi_zones':df_taxi_zones, 'location_id':['PULocationID','DOLocationID']}

In [62]:
def process_timestamps(df):
    
    #We process the datetime, trip diration and weekday name for PU and DO timestamps
    pu_time = split_date_time(clean_up_dict['tpep_PU'])
    do_time = split_date_time(clean_up_dict['tpep_DO'])
    trip_duration = trip_duration_calc(pu_time, do_time)
    pu_trip_weekday = weekday_name_identify(pu_time)
    do_trip_weekday = weekday_name_identify(do_time)

    #We write the new columns to df
    df['PU_datetime'] = pu_time
    df['DO_datetime'] = do_time
    df['trip_duration'] = trip_duration
    df['PU_weekday'] = pu_trip_weekday
    df['DO_weekday'] = do_trip_weekday
    
    return df


def split_date_time(ts_serie):
    #converts the string to datetime format
    ts_datetime = pd.to_datetime(ts_serie)
    
    return ts_datetime

def trip_duration_calc(pu_time, do_time):
    # computes the duration of the trip in seconds
    trip_duration = do_time - pu_time
    trip_duration = trip_duration/np.timedelta64(1,'s')
    
    return trip_duration

def weekday_name_identify(trip_date):
    # returns the name of the weekday of the date of the trip provided
    trip_weekday = trip_date.dt.day_name()

    return trip_weekday

df = process_timestamps(df)
df.drop(columns=['tpep_pickup_datetime','tpep_dropoff_datetime','store_and_fwd_flag'],inplace=True)

In [63]:
def neg_val_treat(df, features):
    
    #for each columns listed as an input, we drop the rows that have negative values
    for feature in features:
        neg_condition = df[df[feature] < 0].index
        df = df.drop(neg_condition)

    return df

########################################################################

def handle_no_passenger(df):
  
    #We create two dataframes, one with only the trips with no passengers, the other with passengers
    no_pass_condition = df[df['passenger_count'] == 0].index
    df = df.drop(no_pass_condition)
    
    return df

neg_features = clean_up_dict['neg_features']
df = neg_val_treat(df, neg_features)
df = handle_no_passenger(df)

In [64]:
new_dict = {'categ_features':['RatecodeID', 'payment_type','PU_weekday', 'DO_weekday']}   
cat = new_dict['categ_features']

#Encode categorical variables
def cat_encoder(df, cat_features):    
    for col in cat_features:
        df[col] = value_integrity(df[col], col)
        df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_')], axis=1)
    return df

def value_integrity(df, cat):
    if cat == 'PU_weekday' or cat == 'DO_weekday':
        expected_val = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']
    else:
        expected_val = [1, 2, 3, 4, 5, 6]
    df = df[df.isin(expected_val)]

    return df

df = cat_encoder(df,cat)

In [65]:
def borough_zone_names(df, taxi_zones):
    taxi_zones = clean_up_dict['taxi_zones']
    #We merge the df with the taxi_zones using the PU location ID first
    df_intermediate = pd.merge(df, taxi_zones, how='inner', left_on='PULocationID', right_on='LocationID')
    
    #We merge the previous df with the taxi_zonesobtained now using the DO location ID
    df_merged = pd.merge(df_intermediate, taxi_zones, how='inner', left_on='DOLocationID', right_on='LocationID', suffixes=('_PU','_DO'))
    
    #We remove unused columns
    unused_columns = ['LocationID_PU', 'LocationID_DO']
    df_merged.drop(unused_columns, axis=1, inplace=True)
    
    #We add new columns to our initial df
    df['PU_borough_name'] = df_merged['Borough_PU']
    df['PU_zone_name'] = df_merged['Zone_PU']
    df['PU_service_zone_name'] = df_merged['service_zone_PU']
    df['DO_borough_name'] = df_merged['Borough_DO']
    df['DO_zone_name'] = df_merged['Zone_DO']
    df['DO_service_zone_name'] = df_merged['service_zone_DO']
    
    return df

df = borough_zone_names(df, clean_up_dict['taxi_zones'])
df = df[df['PU_borough_name'].notna()]

In [66]:
df['total_initial_fare'] = df['fare_amount'] + df['extra'] + df['mta_tax'] + df['tolls_amount'] + df['improvement_surcharge']
df.reset_index(drop=True)

,VendorID,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,DO_weekday_Thursday,DO_weekday_Tuesday,DO_weekday_Wednesday,PU_borough_name,PU_zone_name,PU_service_zone_name,DO_borough_name,DO_zone_name,DO_service_zone_name,total_initial_fare
0,1,2,2.5,148,234,12.0,0.5,0.5,3.95,0.0,...,0,0,0,Manhattan,Lower East Side,Yellow Zone,Manhattan,Union Sq,Yellow Zone,13.3
1,1,3,2.3,170,144,13.0,0.5,0.5,2.85,0.0,...,0,0,0,Manhattan,Lower East Side,Yellow Zone,Manhattan,Union Sq,Yellow Zone,14.3
2,2,1,0.0,113,193,2.5,0.5,0.5,0.00,0.0,...,0,0,0,Manhattan,Lower East Side,Yellow Zone,Manhattan,Union Sq,Yellow Zone,3.8
3,1,1,3.9,95,92,12.5,0.5,0.5,2.75,0.0,...,0,0,0,Manhattan,Lower East Side,Yellow Zone,Manhattan,Union Sq,Yellow Zone,13.8
4,1,1,12.8,163,228,45.0,0.5,0.5,9.25,0.0,...,0,0,0,Manhattan,Lower East Side,Yellow Zone,Manhattan,Union Sq,Yellow Zone,46.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7577194,1,1,1.5,162,107,8.0,0.0,0.5,1.00,0.0,...,0,0,0,Manhattan,Penn Station/Madison Sq West,Yellow Zone,Queens,Jamaica Bay,Boro Zone,8.8
7577195,1,1,1.3,107,144,9.5,0.0,0.5,1.00,0.0,...,0,0,0,Queens,Queensbridge/Ravenswood,Boro Zone,Queens,Jamaica Bay,Boro Zone,10.3
7577196,1,2,1.4,163,170,7.5,0.0,0.5,1.65,0.0,...,0,0,0,Queens,Jackson Heights,Boro Zone,Queens,Jamaica Bay,Boro Zone,8.3
7577197,1,2,1.3,234,161,8.5,0.0,0.5,0.00,0.0,...,0,0,0,Queens,Woodside,Boro Zone,Queens,Jamaica Bay,Boro Zone,9.3


In [67]:
#Encode categorical variables
def cat_encoder_2(df, cat_features):    
    for col in cat_features:
        df[col] = value_integrity_2(df[col], col)
        df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_')], axis=1)
    return df

def value_integrity_2(df, cat):
    if cat == 'PU_borough_name' or cat == 'DO_borough_name':
        expected_val = ['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'EWR','Staten Island']
        
    df = df[df.isin(expected_val)]

    return df

new_dict_2 = {'categ_features':['PU_borough_name', 'DO_borough_name']}   
cat_2 = new_dict_2['categ_features']
df = cat_encoder_2(df, cat_2)

# Save to S3

In [68]:
mybucket = 's3://nyc-tlc-trip-2018/nyc-2018-yellow/data-prep-yellow-2018'  # data_stage/  , if have / in the end, they will create the new folder name '/' and save file into those destination
key = "df_dec_yellow_2018.csv"

df.to_csv(f"s3://{mybucket}/{key}",index=False)